# OpenAI function calling in LangChain

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
import typing as ty
from pydantic import BaseModel, Field

## Pydantic to OpenAI function definition

In [4]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [5]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [6]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [7]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code',
    'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [8]:
from langchain.chat_models import ChatOpenAI

In [9]:
model = ChatOpenAI()

In [12]:
model.invoke("what is the weather in New Delhi?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "DEL"\n}'}})

In [13]:
model_with_function = model.bind(functions=[weather_function])

In [14]:
model_with_function.invoke("what is the weather in New Delhi today?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "DEL"\n}'}})

### Forcing to use a function

In [15]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name": "WeatherSearch"})

In [16]:
model_with_forced_function.invoke("Hello!")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "JFK"\n}'}})

## Using it in a chain

In [17]:
from langchain.prompts import ChatPromptTemplate

In [20]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [21]:
chain = prompt | model_with_function

In [22]:
chain.invoke({"input": "What is the weather in New Delhi"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "DEL"\n}'}})

### Using multiple functions

In [23]:
class GameSearch(BaseModel):
    """Call this to get the names of games by genre"""
    genre: str = Field(description="genre of the game to look up")
    n: int = Field(description="number of results to return")

In [24]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(GameSearch)
]

In [25]:
model_with_functions = model.bind(functions=functions)

In [26]:
model_with_functions.invoke("what is the weather in New Delhi?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "DEL"\n}'}})

In [27]:
model_with_functions.invoke("Give me 5 racing games")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'GameSearch', 'arguments': '{\n  "genre": "racing",\n  "n": 5\n}'}})

In [28]:
model_with_functions.invoke("Hello!")

AIMessage(content='Hi there! How can I assist you today?')